In [2]:
import torch

import os

import torch
import torchvision
from torch import nn , optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from PIL import Image

In [3]:
import math
irange = range

In [4]:
def make_grid(tensor, nrow=8, padding=2,
              normalize=False, range=None, scale_each=False, pad_value=0):
    """Make a grid of images.
    Args:
        tensor (Tensor or list): 4D mini-batch Tensor of shape (B x C x H x W)
            or a list of images all of the same size.
        nrow (int, optional): Number of images displayed in each row of the grid.
            The Final grid size is (B / nrow, nrow). Default is 8.
        padding (int, optional): amount of padding. Default is 2.
        normalize (bool, optional): If True, shift the image to the range (0, 1),
            by subtracting the minimum and dividing by the maximum pixel value.
        range (tuple, optional): tuple (min, max) where min and max are numbers,
            then these numbers are used to normalize the image. By default, min and max
            are computed from the tensor.
        scale_each (bool, optional): If True, scale each image in the batch of
            images separately rather than the (min, max) over all images.
        pad_value (float, optional): Value for the padded pixels.
    Example:
        See this notebook `here <https://gist.github.com/anonymous/bf16430f7750c023141c562f3e9f2a91>`_
    """
    if not (torch.is_tensor(tensor) or
            (isinstance(tensor, list) and all(torch.is_tensor(t) for t in tensor))):
        raise TypeError('tensor or list of tensors expected, got {}'.format(type(tensor)))

    # if list of tensors, convert to a 4D mini-batch Tensor
    if isinstance(tensor, list):
        tensor = torch.stack(tensor, dim=0)

    if tensor.dim() == 2:  # single image H x W
        tensor = tensor.view(1, tensor.size(0), tensor.size(1))
    if tensor.dim() == 3:  # single image
        if tensor.size(0) == 1:  # if single-channel, convert to 3-channel
            tensor = torch.cat((tensor, tensor, tensor), 0)
        tensor = tensor.view(1, tensor.size(0), tensor.size(1), tensor.size(2))

    if tensor.dim() == 4 and tensor.size(1) == 1:  # single-channel images
        tensor = torch.cat((tensor, tensor, tensor), 1)

    if normalize is True:
        tensor = tensor.clone()  # avoid modifying tensor in-place
        if range is not None:
            assert isinstance(range, tuple), \
                "range has to be a tuple (min, max) if specified. min and max are numbers"

        def norm_ip(img, min, max):
            img.clamp_(min=min, max=max)
            img.add_(-min).div_(max - min + 1e-5)

        def norm_range(t, range):
            if range is not None:
                norm_ip(t, range[0], range[1])
            else:
                norm_ip(t, float(t.min()), float(t.max()))

        if scale_each is True:
            for t in tensor:  # loop over mini-batch dimension
                norm_range(t, range)
        else:
            norm_range(tensor, range)

    if tensor.size(0) == 1:
        return tensor.squeeze()

    # make the mini-batch of images into a grid
    nmaps = tensor.size(0)
    xmaps = min(nrow, nmaps)
    ymaps = int(math.ceil(float(nmaps) / xmaps))
    height, width = int(tensor.size(2) + padding), int(tensor.size(3) + padding)
    grid = tensor.new(3, height * ymaps + padding, width * xmaps + padding).fill_(pad_value)
    k = 0
    for y in irange(ymaps):
        for x in irange(xmaps):
            if k >= nmaps:
                break
            grid.narrow(1, y * height + padding, height - padding)\
                .narrow(2, x * width + padding, width - padding)\
                .copy_(tensor[k])
            k = k + 1
    return grid


def save_image(tensor, filename, nrow=8, padding=2,
               normalize=False, range=None, scale_each=False, pad_value=0):
    """Save a given Tensor into an image file.
    Args:
        tensor (Tensor or list): Image to be saved. If given a mini-batch tensor,
            saves the tensor as a grid of images by calling ``make_grid``.
        **kwargs: Other arguments are documented in ``make_grid``.
    """
    from PIL import Image
    grid = make_grid(tensor, nrow=nrow, padding=padding, pad_value=pad_value,
                     normalize=normalize, range=range, scale_each=scale_each)
    ndarr = grid.mul(255).clamp(0, 255).byte().permute(1, 2, 0).cpu().numpy()
    im = Image.fromarray(ndarr)
    im.save(filename)

In [5]:
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.24703223,  0.24348513 , 0.26158784))
])

trainset = datasets.CIFAR10(root='./data', train=True,download=True, transform=img_transform)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=16,
                                          shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,download=True, transform=img_transform)

test_loader = torch.utils.data.DataLoader(testset, batch_size=16,shuffle=False, num_workers=2)

100%|██████████| 170498071/170498071 [00:02<00:00, 79602108.00it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [35]:
CHANNELS = 3
HEIGHT = 32
WIDTH = 32
EPOCHS = 30
LOG_INTERVAL = 500

In [7]:
class Interpolate(nn.Module):
    def __init__(self, size, mode):
        super(Interpolate, self).__init__()
        self.interp = nn.functional.interpolate
        self.size = size
        self.mode = mode  
        
    def forward(self, x):
        x = self.interp(x, size=self.size, mode=self.mode, align_corners=False)
        return x

In [8]:
class End_to_end(nn.Module):
    def __init__(self):
        super(End_to_end, self).__init__()
    
        # Encoder
        # TODO : try with padding = 0
        self.conv1 = nn.Conv2d(CHANNELS, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=0)
        self.bn1 = nn.BatchNorm2d(64, affine=False)
        self.conv3 = nn.Conv2d(64, CHANNELS, kernel_size=3, stride=1, padding=1)
        
        # Decoder
        #TODO : try ConvTranspose2d
        self.interpolate = Interpolate(size=HEIGHT, mode='bilinear')
        self.deconv1 = nn.Conv2d(CHANNELS, 64, 3, stride=1, padding=1)
        self.deconv2 = nn.Conv2d(64, 64, 3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64, affine=False)

        self.deconv_n = nn.Conv2d(64, 64, 3, stride=1, padding=1)
        self.bn_n = nn.BatchNorm2d(64, affine=False)


        self.deconv3 = nn.ConvTranspose2d(64, CHANNELS, 3, stride=1, padding=1)

        self.relu = nn.ReLU()
        
    def encode(self, x):
        out = self.relu(self.conv1(x))
        out = self.relu(self.conv2(out))
        out = self.bn1(out)
        return self.conv3(out)
    def reparameterize(self, mu, logvar):
        pass
    
    def decode(self, z):
        upscaled_image = self.interpolate(z)
        out = self.relu(self.deconv1(upscaled_image))
        out = self.relu(self.deconv2(out))
        out = self.bn2(out)
        for _ in range(10):
            out = self.relu(self.deconv_n(out))
            out = self.bn_n(out)
        out = self.deconv3(out)
        final = upscaled_image + out
        return final,out,upscaled_image
    
    def forward(self, x):
        com_img = self.encode(x)
        final,out,upscaled_image = self.decode(com_img)
        return final, out, upscaled_image, com_img, x
    

In [9]:
CUDA = torch.cuda.is_available()
if CUDA:
    model = End_to_end().cuda()
else :
    model = End_to_end()

optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [11]:
def loss_function(final_img,residual_img,upscaled_img,com_img,orig_img):
    com_loss = nn.MSELoss(size_average=False)(orig_img, final_img)
    rec_loss = nn.MSELoss(size_average=False)(residual_img,orig_img-upscaled_img)
    return com_loss + rec_loss

In [36]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = Variable(data)
        optimizer.zero_grad()
        final, residual_img, upscaled_image, com_img, orig_im = model(data.cuda())
        loss = loss_function(final, residual_img, upscaled_image, com_img, orig_im)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
        if batch_idx % LOG_INTERVAL == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))     

In [37]:
def test(epoch):
  
    model.eval()
    test_loss = 0
    for i, (data, _) in enumerate(test_loader):
        data = Variable(data)
        final, residual_img, upscaled_image, com_img, orig_im = model(data.cuda())
        test_loss += loss_function(final, residual_img, upscaled_image, com_img, orig_im).item() * data.size(0)
        if epoch == EPOCHS and i == 0:
#             save_image(final.data[0],'reconstruction_final',nrow=8)
#             save_image(com_img.data[0],'com_img',nrow=8)
            n = min(data.size(0), 6)
            print("saving the image "+str(n))
            comparison = torch.cat([data[:n],
              final[:n].cpu()])
            comparison = comparison.cpu()
#             print(comparison.data)
            save_image(com_img[:n].data,
                       'compressed_' + str(epoch) +'.png', nrow=n)
            save_image(comparison.data,
                       'reconstruction_' + str(epoch) +'.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

End_to_end(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (conv3): Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (interpolate): Interpolate()
  (deconv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (deconv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (deconv_n): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn_n): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (deconv3): ConvTranspose2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU()
)

In [39]:
for epoch in range(1, EPOCHS+1):
    train(epoch)
    test(epoch)
    if epoch == EPOCHS:
        pass
#         sample = Variable(torch.randn(64, args.hidden_size),16)
#         sample = model.decode(sample)
#         print("saving im")
#         save_image(sample.data.view(64, 3, 32, 32),
# 'sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/50000 (0%)]	Loss: 126.118851
Train Epoch: 1 [8000/50000 (16%)]	Loss: 98.546265
Train Epoch: 1 [16000/50000 (32%)]	Loss: 177.142273
Train Epoch: 1 [24000/50000 (48%)]	Loss: 114.054276
Train Epoch: 1 [32000/50000 (64%)]	Loss: 117.828644
Train Epoch: 1 [40000/50000 (80%)]	Loss: 146.150360
Train Epoch: 1 [48000/50000 (96%)]	Loss: 91.173653
====> Epoch: 1 Average loss: 2067.9632
====> Test set loss: 6675.7213
Train Epoch: 2 [0/50000 (0%)]	Loss: 124.630196
Train Epoch: 2 [8000/50000 (16%)]	Loss: 109.089935
Train Epoch: 2 [16000/50000 (32%)]	Loss: 115.094406
Train Epoch: 2 [24000/50000 (48%)]	Loss: 104.540436
Train Epoch: 2 [32000/50000 (64%)]	Loss: 142.997589
Train Epoch: 2 [40000/50000 (80%)]	Loss: 123.259140
Train Epoch: 2 [48000/50000 (96%)]	Loss: 134.290833
====> Epoch: 2 Average loss: 2026.5839
====> Test set loss: 7732.0445
Train Epoch: 3 [0/50000 (0%)]	Loss: 158.209778
Train Epoch: 3 [8000/50000 (16%)]	Loss: 139.417999
Train Epoch: 3 [16000/50000 (32%)]	Loss: 83.7380

In [40]:
torch.save(model.state_dict(), './net.pth')

In [41]:
model.load_state_dict(torch.load('net.pth'))

<All keys matched successfully>

In [45]:
def save_images():
    epoch = EPOCHS
    model.eval()
    test_loss = 0
    for i, (data, _) in enumerate(test_loader):
        data = Variable(data)
        final, residual_img, upscaled_image, com_img, orig_im = model(data.cuda())
        test_loss += loss_function(final, residual_img, upscaled_image, com_img, orig_im).item() * data.size(0)
        if i == 3:
#             save_image(final.data[0],'reconstruction_final',nrow=8)
#             save_image(com_img.data[0],'com_img',nrow=8)
            n = min(data.size(0), 6)
            print("saving the image "+str(n))
            comparison = torch.cat([data[:n],
              final[:n].cpu()])
            comparison = comparison.cpu()
#             print(comparison.data)
            save_image(com_img[:1].data,
                         'compressed_' + str(i) +'.png', nrow=n)
            save_image(final[:1].data,
                        'final_' + str(epoch) +'.png', nrow=n)
            save_image(orig_im[:1].data,
                        'original_' + str(epoch) +'.png', nrow=n)
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [46]:
save_images()

saving the image 6
====> Test set loss: 5765.6688
